In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'numpy'

In [ ]:
titanic_data = pd.read_csv('sample_data/train.csv')

In [ ]:
titanic_data.head()

In [ ]:
titanic_data.shape

In [ ]:
titanic_data.info()

In [ ]:
titanic_data.isnull().sum()

In [ ]:
titanic_data = titanic_data.drop(columns='Cabin', axis=1)

In [ ]:
titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace=True)

In [ ]:
titanic_data.info()

In [ ]:
titanic_data.isnull().sum()

In [ ]:
print(titanic_data['Embarked'].mode())

In [ ]:
print(titanic_data['Embarked'].mode()[0])

In [ ]:
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace=True)

In [ ]:
titanic_data.isnull().sum()

## **Analysing the Data**

In [ ]:
titanic_data.describe()

In [ ]:
titanic_data['Survived'].value_counts()

In [ ]:
sns.set()

In [ ]:
sns.countplot(titanic_data['Survived'])

In [ ]:
titanic_data['Sex'].value_counts()

In [ ]:
sns.countplot(titanic_data['Sex'])

In [ ]:
sns.countplot(x='Sex', hue='Survived', data=titanic_data)


In [ ]:
sns.countplot(x='Pclass', data=titanic_data)

In [ ]:
sns.countplot(x='Pclass', hue='Survived', data=titanic_data)

## **Encode categorical columns/data**

In [ ]:
titanic_data['Sex'].value_counts()

In [ ]:
titanic_data['Embarked'].value_counts()

In [ ]:
titanic_data.replace({'Sex':{'male':0,'female':1}, 'Embarked':{'S':0,'C':1,'Q':2}})


In [ ]:
X = titanic_data.drop(columns = ['PassengerId','Name','Ticket','Survived'],axis=1)
Y = titanic_data['Survived']

In [ ]:
print(X)

In [ ]:
print(Y)

## **Split the data into test data and train data**

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

## **Evaluating and testing the model**

In [ ]:
X_train_prediction = model.predict(X_train)

In [ ]:
print(X_train_prediction)

In [ ]:
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score of training data : ', training_data_accuracy)

In [ ]:
X_test_prediction = model.predict(X_test)

In [ ]:
print(X_test_prediction)

In [ ]:
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score of test data : ', test_data_accuracy)

In [ ]:
import joblib
joblib.dump(model, 'logistic_regression_model.pkl')

In [ ]:
!pip install pyngrok

import subprocess
import os
from pyngrok import ngrok

ngrok.set_auth_token("2l62uIQqv8m7YmkUUBLMfq9xgeN_Di9PyFXgXCMFwtQ11WAV")
os.system("nohup python -m flask run --no-reload &")
proc = subprocess.Popen(['ngrok', 'http', '5000'])
public_url = ngrok.connect(addr="5000", proto="http")
print("Public URL:", public_url)

In [ ]:
from flask import Flask, request, jsonify
import joblib
from pyngrok import ngrok
from IPython.display import display, HTML

model = joblib.load('logistic_regression_model.pkl')
app = Flask(__name__)
@app.route('/')
def home():
  html_form = """
  <!DOCTYPE html>
  <html>
  <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Titanic Survival Prediction</title>
  </head>
  <body>
    <h2>Titanic Survival Prediction</h2>
    <form id="predictionForm method="POST" action="/predict">
    <label for="pclass">Pclass:</label>
    <input type="text" id="pclass" name="pclass" required><br><br>

    <label for="sex">Sex(0 for male, 1 for female):</label>
    <input type="text" id="sex" name="sex" required><br><br>

    <label for="age">Age:</label>
    <input type="text" id="age" name="age" required><br><br>

    <label for="sibsp">Sibsp:</label>
    <input type="text" id="sibsp" name="sibsp" required><br><br>

    <label for="parch">Parch:</label>
    <input type="text" id="parch" name="parch" required><br><br>

    <label for="fare">Fare:</label>
    <input type="text" id="fare" name="fare" required><br><br>

    <label for="embarked">Embarked(0 for S, 1 for C, 2 for Q):</label>
    <input type="text" id="embarked" name="embarked" required><br><br>

    <button type="button" onclick="predictSurvival()">Predict</button>
    </form>

    <p id="predictionResult"></p>

    <script>
    function predictSurvival() {
      var xhr = new XMLHttpRequest();
      var url = "/predict";
      var data = new FormData(document.getElementById("predictionForm"));

      xhr.open("POST", url, true);
      xhr.onreadystatechange = function() {
        if (xhr.readyState === 4 && xhr.status === 200) {
          var result = xhr.responseText;
          document.getElementById("predictionResult").textContent = result;
        }
        };
      xhr.send(data);
    }
    </script>
  </body>
  </html>
  """
  return html_form

@app.route('/predict', methods=['POST'])
def predict():
    try:
        pclass = int(request.form['pclass'])
        sex = int(request.form['sex'])
        age = float(request.form['age'])
        sibsp = int(request.form['sibsp'])
        parch = int(request.form['parch'])
        fare = float(request.form['fare'])
        embarked = int(request.form['embarked'])

        features = [[pclass, sex, age, sibsp, parch, fare, embarked]]
        print(f"Received features: {features}")  # Debugging print

        prediction = model.predict(features)
        print(f"Model prediction: {prediction}")  # Debugging print

        return jsonify({'prediction': int(prediction[0])})
    except Exception as e:
        print(f"Error in prediction: {e}")
        return jsonify({'error': 'Prediction failed, check input data and model.'})

def run_flask_app():
  app.run(host='127.0.0.1', port=5000, debug=True, use_reloader=False)

public_url = ngrok.connect(addr="5000", proto="http")
print("Public URL:", public_url)
display(HTML(f"<h2>Open this linkn in your browser to accesss the application:</h2><p>{public_url}</p>"))
try:
  run_flask_app()
except:
  ngrok.kill()
